In [1]:
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

In [12]:
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]

    return access_token


consumer_key = "0557bd5252b34142911d"
consumer_secret = "d90d86911924440786d4"

In [2]:
# 서울시 대학교 데이터 리스트 형태로 가지고 오는 함수
def get_university_list() -> None:
    url = "http://openapi.seoul.go.kr:8088/5274694f4f6d6f6e33395743465a61/json/SebcCollegeInfoKor/1/1000/"

    temp_res = requests.get(
        url=url,
        headers={
            "Content-Type": "application/json; chearset=utf-8"
        },
    )

    temp_result_list = temp_res.json()["SebcCollegeInfoKor"]["row"]

    return temp_result_list

In [22]:
temp_seoul_university_list = get_university_list()
seoul_university_df = pd.DataFrame(temp_seoul_university_list)
seoul_university_df.head()

,MAIN_KEY,YEAR,CATE1_NAME,NAME_KOR,BRANCH,STATE,TYPE,POSTCODE,ADD_KOR,ADD_KOR_ROAD,H_KOR_CITY,H_KOR_GU,H_KOR_DONG,TEL,FAX,HP
0,BE_LiST23-0034,2013,일반대학,서울시립대학교,본교,기존,공립,,서울 동대문구 서울시립대로 163 (전농동 90번지),,서울특별시,동대문구,휘경2동,02-6490-6114,02-2210-5576,http://www.uos.ac.kr
1,BE_LiST23-0035,2013,전문대학(3년제),서울여자간호대학교,본교,기존,사립,,서울 서대문구 홍제3동 서울여자간호대학,,서울특별시,서대문구,홍제2동,02-2287-1700,02-395-8018,http://www.snjc.ac.kr
2,BE_LiST23-0036,2013,일반대학,서울여자대학교,본교,기존,사립,,서울특별시 노원구 화랑로 621 서울여자대학교,,서울특별시,노원구,공릉2동,02-970-5114,02-978-7931,http://www.swu.ac.kr
3,BE_LiST23-0037,2013,전문대학(2년제),서일대학교,본교,학교명 변경,사립,,서울 중랑구 서일대학길 22(면목동 49-3) 서일대학교,,서울특별시,중랑구,면목제3.8동,02-490-7300,02-493-2576,http://www.seoil.ac.kr
4,BE_LiST23-0038,2013,일반대학,성공회대학교,본교,기존,사립,,서울 구로구 항동 성공회대학교,,서울특별시,구로구,오류2동,02-2610-4142,02-2610-4248,http://www.skhu.ac.kr


In [23]:
seoul_university_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   MAIN_KEY      64 non-null     object
 1   YEAR          64 non-null     object
 2   CATE1_NAME    64 non-null     object
 3   NAME_KOR      64 non-null     object
 4   BRANCH        64 non-null     object
 5   STATE         64 non-null     object
 6   TYPE          64 non-null     object
 7   POSTCODE      64 non-null     object
 8   ADD_KOR       64 non-null     object
 9   ADD_KOR_ROAD  64 non-null     object
 10  H_KOR_CITY    64 non-null     object
 11  H_KOR_GU      64 non-null     object
 12  H_KOR_DONG    64 non-null     object
 13  TEL           64 non-null     object
 14  FAX           64 non-null     object
 15  HP            64 non-null     object
dtypes: object(16)
memory usage: 8.1+ KB


In [24]:
seoul_university_df["STATE"].unique()

array(['기존', '학교명 변경', '폐교', '학제 변경'], dtype=object)

In [25]:
# 폐교 상태인 학교는 제외
seoul_university_df = seoul_university_df[seoul_university_df['STATE'] != '폐교']
seoul_university_df['STATE'].unique()

array(['기존', '학교명 변경', '학제 변경'], dtype=object)

In [26]:
seoul_university_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 0 to 63
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   MAIN_KEY      62 non-null     object
 1   YEAR          62 non-null     object
 2   CATE1_NAME    62 non-null     object
 3   NAME_KOR      62 non-null     object
 4   BRANCH        62 non-null     object
 5   STATE         62 non-null     object
 6   TYPE          62 non-null     object
 7   POSTCODE      62 non-null     object
 8   ADD_KOR       62 non-null     object
 9   ADD_KOR_ROAD  62 non-null     object
 10  H_KOR_CITY    62 non-null     object
 11  H_KOR_GU      62 non-null     object
 12  H_KOR_DONG    62 non-null     object
 13  TEL           62 non-null     object
 14  FAX           62 non-null     object
 15  HP            62 non-null     object
dtypes: object(16)
memory usage: 8.2+ KB


In [27]:
# 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key, consumer_secret)
    url = "https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params = {"accessToken": access_token, "address": address}

    temp_res = requests.get(url=url, params=params).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

In [28]:
for _, row in seoul_university_df.iterrows():  # iterrows는 데이터프레임에서 각 행을 순회, 사용법은 row[칼럼명] 을 하면 각 행당 칼럼명에 해당하는 값만 추출
    try:
        x, y = get_adress_x_y(row["ADD_KOR"])

        seoul_university_df.loc[_, "x"] = x
        seoul_university_df.loc[_, "y"] = y
    except:
        print(row["ADD_KOR"] + '은(는) 검색에 실패했습니다')

seoul_university_df.head()

서울 종로구 명륜동3가 성균관대학교은(는) 검색에 실패했습니다
서울 노원구 상계7동 한국성서대학교은(는) 검색에 실패했습니다
서울 성북구 삼선동3가 389 한성대학교은(는) 검색에 실패했습니다
서울 성북구 하월곡2동 동덕여자대학교은(는) 검색에 실패했습니다


,MAIN_KEY,YEAR,CATE1_NAME,NAME_KOR,BRANCH,STATE,TYPE,POSTCODE,ADD_KOR,ADD_KOR_ROAD,H_KOR_CITY,H_KOR_GU,H_KOR_DONG,TEL,FAX,HP,x,y
0,BE_LiST23-0034,2013,일반대학,서울시립대학교,본교,기존,공립,,서울 동대문구 서울시립대로 163 (전농동 90번지),,서울특별시,동대문구,휘경2동,02-6490-6114,02-2210-5576,http://www.uos.ac.kr,960760.843699103,1953973.31776084
1,BE_LiST23-0035,2013,전문대학(3년제),서울여자간호대학교,본교,기존,사립,,서울 서대문구 홍제3동 서울여자간호대학,,서울특별시,서대문구,홍제2동,02-2287-1700,02-395-8018,http://www.snjc.ac.kr,951546.69437466003,1954865.33199999994
2,BE_LiST23-0036,2013,일반대학,서울여자대학교,본교,기존,사립,,서울특별시 노원구 화랑로 621 서울여자대학교,,서울특별시,노원구,공릉2동,02-970-5114,02-978-7931,http://www.swu.ac.kr,964088.33042111,1958616.77872277
3,BE_LiST23-0037,2013,전문대학(2년제),서일대학교,본교,학교명 변경,사립,,서울 중랑구 서일대학길 22(면목동 49-3) 서일대학교,,서울특별시,중랑구,면목제3.8동,02-490-7300,02-493-2576,http://www.seoil.ac.kr,964473.213136262,1954193.75976133
4,BE_LiST23-0038,2013,일반대학,성공회대학교,본교,기존,사립,,서울 구로구 항동 성공회대학교,,서울특별시,구로구,오류2동,02-2610-4142,02-2610-4248,http://www.skhu.ac.kr,940400.145153242978,1942624.71430000011


### UTM-K이므로 WSG84좌표계로 변환

In [29]:
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 key를 입력

In [30]:
for _, row in seoul_university_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])

        seoul_university_df.loc[_, "lng"] = x
        seoul_university_df.loc[_, "lat"] = y

    except:
        pass
seoul_university_df.head()

,MAIN_KEY,YEAR,CATE1_NAME,NAME_KOR,BRANCH,STATE,TYPE,POSTCODE,ADD_KOR,ADD_KOR_ROAD,H_KOR_CITY,H_KOR_GU,H_KOR_DONG,TEL,FAX,HP,x,y,lng,lat
0,BE_LiST23-0034,2013,일반대학,서울시립대학교,본교,기존,공립,,서울 동대문구 서울시립대로 163 (전농동 90번지),,서울특별시,동대문구,휘경2동,02-6490-6114,02-2210-5576,http://www.uos.ac.kr,960760.843699103,1953973.31776084,127.055569,37.584315
1,BE_LiST23-0035,2013,전문대학(3년제),서울여자간호대학교,본교,기존,사립,,서울 서대문구 홍제3동 서울여자간호대학,,서울특별시,서대문구,홍제2동,02-2287-1700,02-395-8018,http://www.snjc.ac.kr,951546.69437466003,1954865.33199999994,126.951153,37.591916
2,BE_LiST23-0036,2013,일반대학,서울여자대학교,본교,기존,사립,,서울특별시 노원구 화랑로 621 서울여자대학교,,서울특별시,노원구,공릉2동,02-970-5114,02-978-7931,http://www.swu.ac.kr,964088.33042111,1958616.77872277,127.093028,37.626303
3,BE_LiST23-0037,2013,전문대학(2년제),서일대학교,본교,학교명 변경,사립,,서울 중랑구 서일대학길 22(면목동 49-3) 서일대학교,,서울특별시,중랑구,면목제3.8동,02-490-7300,02-493-2576,http://www.seoil.ac.kr,964473.213136262,1954193.75976133,127.097605,37.586453
4,BE_LiST23-0038,2013,일반대학,성공회대학교,본교,기존,사립,,서울 구로구 항동 성공회대학교,,서울특별시,구로구,오류2동,02-2610-4142,02-2610-4248,http://www.skhu.ac.kr,940400.145153242978,1942624.71430000011,126.825893,37.480936


In [31]:
center = [37.541, 126.986]
map = folium.Map(location=center, zoom_start=12)

In [32]:
for _, row in seoul_university_df.iterrows():
    try:
        folium.Marker(
            location=[row["lat"], row["lng"]], # 좌표
            popup=row["NAME_KOR"], # 클릭시 표시될 popup 내용
            icon=folium.Icon(color="cadetblue", icon="info-sign") # 표시될 아이콘 설정
        ).add_to(map)
    except:
        pass

In [33]:
map